<a href=\"https://colab.research.google.com/github/Samir-atra/CancerDetector/blob/main/Meningioma_Detector/Cancer_Detector_visualization.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>

In [10]:
# 1. Imports and Setup
# --------------------
# This cell installs and imports the necessary libraries.
# It also logs into Weights & Biases (wandb) for experiment tracking.

!pip install keras_tuner -q
!pip install wandb -qU

import tensorflow as tf
import keras_tuner
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import sys
import time
import wandb
from wandb.keras import WandbMetricsLogger

# Log in to Weights & Biases to track experiments.
wandb.login()

\u001b\u001bwandb\u001b: \u001bWARNING\u001b Calling wandb.login() after wandb.init() has no effect.\n

True

In [11]:
# 2. Mount Google Drive
# ---------------------
# Mount Google Drive to access the dataset, which is stored there.
# This is necessary when running the notebook in Google Colab.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount(\"/content/drive\", force_remount=True).\n

In [12]:
# 3. Initialize Weights & Biases
# ------------------------------
# Initialize a new wandb run to log metrics for this experiment.

wandb.init(
    project="cancer_detector",
    config={
        "epochs": 10,
        "batch_size": 5,
        "lr": 0.0005670323757088328
    })

# Create a config object for easy access to hyperparameters.
config = wandb.config

In [13]:
# 4. Dataset Loading
# ------------------
# Load the training and validation datasets from Google Drive.

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')
data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
AUTOTUNE = tf.data.AUTOTUNE

start = time.time()
dataset_path, dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels='inferred',
    validation_split=0.2,
    subset='both',
    seed=1,
    batch_size=config.batch_size, # Use batch size from wandb config.
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)

# The testing dataset is commented out as it is not used in this notebook.
# dataset_path_test = tf.keras.utils.image_dataset_from_directory(
#     data_path_test,
#     labels='inferred',
#     seed=3,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)

end = time.time()
print(f"Time elapsed for data loading: {end - start:.2f} seconds")


Found 4096 files belonging to 2 classes.\nUsing 3277 files for training.\nFound 4096 files belonging to 2 classes.\nUsing 819 files for validation.\ntime elapsed =  3.7418885231018066\n

In [14]:
# 5. Performance Optimization (Commented Out)
# ------------------------------------------
# Caching and prefetching can be enabled for faster training.

# AUTOTUNE = tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [15]:
# 6. Load Base Model
# ------------------
# Download the InceptionV3 model, pre-trained on ImageNet, without its top classification layer.

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(299, 299, 3),
    include_top=False,
    weights="imagenet")

In [16]:
# 7. Save Base Model
# ------------------
# Save the downloaded base model to Google Drive to avoid re-downloading.

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/SavedBaseModel.h5')
base_model.save(saving_path)

In [ ]:
# 8. Model Definition and Training with WandB
# -------------------------------------------
# This cell defines the full model, compiles it, and trains it while logging metrics to WandB.

model_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/SavedBaseModel.h5')
base_model = tf.keras.models.load_model(model_path)
base_model.trainable = False # Freeze the base model for feature extraction.

# Define preprocessing and augmentation layers.
rescaling = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1) # Rescale to [-1, 1] for InceptionV3.
])
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)
])

# Build the final model.
inputs = tf.keras.Input(shape=(299, 299, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model with the learning rate from the wandb config.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.lr),
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly=True)

# Train the model and log metrics to WandB.
model.fit(
    dataset_path,
    epochs=config.epochs,
    validation_data=dataset_path_val,
    callbacks=[WandbMetricsLogger(log_freq=10)]) # Use WandbMetricsLogger callback.

model.summary()

# Finish the wandb run.
wandb.finish()

# The following sections for fine-tuning are commented out.
# Fine-tuning would involve unfreezing the base model and training with a lower learning rate.


Epoch 1/10\n656/656 [==============================] - 271s 412ms/step - loss: 0.2462 - accuracy: 0.9017 - val_loss: 0.1821 - val_accuracy: 0.9341\nEpoch 2/10\n656/656 [==============================] - 267s 407ms/step - loss: 0.1612 - accuracy: 0.9408 - val_loss: 0.2477 - val_accuracy: 0.9182\nEpoch 3/10\n656/656 [==============================] - 267s 407ms/step - loss: 0.1295 - accuracy: 0.9530 - val_loss: 0.1454 - val_accuracy: 0.9475\nEpoch 4/10\n656/656 [==============================] - 268s 409ms/step - loss: 0.1157 - accuracy: 0.9594 - val_loss: 0.1346 - val_accuracy: 0.9487\nEpoch 5/10\n656/656 [==============================] - 268s 409ms/step - loss: 0.1105 - accuracy: 0.9643 - val_loss: 0.1279 - val_accuracy: 0.9487\nEpoch 6/10\n656/656 [==============================] - 267s 406ms/step - loss: 0.0972 - accuracy: 0.9646 - val_loss: 0.1234 - val_accuracy: 0.9536\nEpoch 7/10\n127/656 [====>.........................] - ETA: 3:07 - loss: 0.0996 - accuracy: 0.9654

In [ ]:
# This cell appears to be a scratchpad for trying out different wandb configurations.

# for _ in range(5):
#     # 🐝 initialise a wandb run
#     wandb.init(
#         project="cancer_detector",
#         config={
#             "epochs": 10,
#             "batch_size": 5,
#             "lr": 1e-3
#             })
#
#     # Copy your config
#     config = wandb.config
#
#     # Dataset_path
#     # Dataset_path_val
#     # model